In [2]:
#!pip install facenet-pytorch

In [5]:
import os
import argparse
from PIL import Image
import torch
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, MTCNN

In [6]:
def load_image(path, mtcnn, transform):
    """Загружает и обрабатывает изображение лица."""
    img = Image.open(path).convert('RGB')
    # Выделение лица
    face = mtcnn(img)
    if face is None:
        raise ValueError(f"Лицо не обнаружено на изображении {path}")
    # mtcnn возвращает уже тензор (3xHxW)
    return face.unsqueeze(0)

In [7]:
def compute_embedding(model, face_tensor):
    """Вычисляет эмбеддинг лица."""
    model.eval()
    with torch.no_grad():
        embedding = model(face_tensor)
    return embedding / embedding.norm(p=2)

# Файлы с собственными изображениями должны содержать "self" в названии

In [12]:
# Инициализация MTCNN и модели FaceNet
mtcnn = MTCNN(image_size=160, margin=0)
model = InceptionResnetV1(pretrained="vggface2").eval()

In [14]:
ref = "./self1.jpg" # Путь до референснового изображения
tests = "./tests" # Путь до тестовых изображений
threshold = 0.5 # Порог косинусного расстояния для совпадения

# Загрузка и обработка референса
ref_face = load_image(ref, mtcnn, None)
ref_emb = compute_embedding(model, ref_face)

# Обработка тестовых
results = []
for fname in sorted(os.listdir(tests)):
    path = os.path.join(tests, fname)
    try:
        face = load_image(path, mtcnn, None)
        emb = compute_embedding(model, face)
        # косинусное расстояние
        dist = 1 - (emb @ ref_emb.T).item()
        match = dist < threshold
        results.append((fname, dist, match))
    except Exception as e:
        # results.append((fname, None, False))
        print(f"Ошибка обработки {fname}: {e}")

# Итог
print("Результаты распознавания:")
other_correct = 0
self_correct = 0
self_count = 0
for fname, dist, match in results:
    label = "MATCH" if match else "NO MATCH"
    dist_str = f"{dist:.3f}" if dist is not None else "N/A"
    print(f"{fname}: {label} (dist={dist_str})")
    # Предполагаем, что имена файлов содержат "self" для своих
    if "self" in fname:
        self_count += 1
        if match:
            self_correct += 1
    else:
        if not match:
            other_correct += 1

other_count = len(results) - self_count
print(f"\nУспешных \"не вы\": {other_correct} из {other_count}")
print(f"Успешных \"вы\": {self_correct} из {self_count}")

assert other_correct / other_count >= 0.6, "Требование: минимум 60% чужих лиц распознано правильно"
assert self_correct / self_count >= 0.6, "Требование: минимум 60% собственных лиц распознано правильно"

print("Минимальные требования выполнены")

Результаты распознавания:
!self2.jpg: MATCH (dist=0.238)
!self3.jpg: MATCH (dist=0.235)
!self4.jpg: MATCH (dist=0.450)
!self5.jpg: MATCH (dist=0.292)
!self6.jpg: MATCH (dist=0.259)
!self7.jpg: MATCH (dist=0.279)
aaron_eckhart.jpg: NO MATCH (dist=0.825)
angelina_jolie.jpg: NO MATCH (dist=1.069)
bradley_cooper.jpg: NO MATCH (dist=0.984)
kate_siegel.jpg: NO MATCH (dist=1.045)
logan_marshall.jpg: NO MATCH (dist=0.756)
paul_rudd.jpg: NO MATCH (dist=0.974)
robert_downey.jpg: NO MATCH (dist=1.060)
ryan_gosling.jpg: NO MATCH (dist=1.068)
shea_whigham.jpg: NO MATCH (dist=1.060)
tom_hardy.jpg: NO MATCH (dist=0.630)
tom_hardy_2.jpg: NO MATCH (dist=0.803)
tom_hardy_3.jpg: NO MATCH (dist=0.802)
tom_hardy_4.jpg: NO MATCH (dist=0.506)
tom_hardy_5.jpg: NO MATCH (dist=0.834)
will_smith.jpg: NO MATCH (dist=1.013)

Успешных "не вы": 15 из 15
Успешных "вы": 6 из 6
Минимальные требования выполнены
